In [9]:

def config_to_moe_args(d_model,hidden_ratio,device):
    from megablocks.layers.arguments import Arguments as MoEArgs


    kwargs = {
        "activation_fn": "swiglu",
        "mlp_type": "mlp",
        "hidden_size": d_model,
        "ffn_hidden_size": d_model*hidden_ratio,
        "moe_num_experts": 4,
        "num_layers": 1,
        # Handled by FSDP (https://github.com/databricks/megablocks/issues/57#issuecomment-1854594483)
        # "moe_weight_parallelism": False,
        "moe_expert_model_parallelism": False,
        "moe_top_k": 1,
        # Handled by FSDP
        "bf16": False,
        "fp16": False,
        "return_bias": False,
    }

    return MoEArgs(**kwargs)

args = config_to_moe_args(2**11,2,"cuda")
args

Arguments(hidden_size=2048, ffn_hidden_size=4096, num_layers=1, bias=True, return_bias=False, activation_fn='swiglu', moe_num_experts=4, moe_top_k=1, moe_capacity_factor=1, moe_normalize_expert_weights=None, moe_loss_weight=0.1, moe_zloss_weight=0.001, moe_jitter_eps=None, moe_lbl_in_fp32=False, moe_expert_choice=False, moe_expert_choice_grouped=False, moe_expert_model_parallelism=False, expert_parallel_group=None, moe_weight_parallelism=False, weight_parallel_group=None, pipeline_model_parallel_size=1, num_layers_per_virtual_pipeline_stage=None, memory_optimized_mlp=False, mlp_type='mlp', mlp_impl='sparse', fp16=False, bf16=False, device=0, init_method=functools.partial(<function normal_ at 0x744347520c20>, mean=0.0, std=0.02), output_layer_init_method=functools.partial(<function normal_ at 0x744347520c20>, mean=0.0, std=0.02), uniform_expert_assignment=False, shared_expert=False, fc_cls=<class 'torch.nn.modules.linear.Linear'>, fc_kwargs={}, remat_act_fn=True, shared_expert_hidden_si

In [10]:
from megablocks.layers.dmoe import dMoE

dmoe = dMoE(args)

# show the number of parameters
print(sum(p.numel() for p in dmoe.parameters()))

# show the number of parameters in the model
print(dmoe)



67119104
dMoE(
  (router): LearnedRouter(
    (layer): Linear(in_features=2048, out_features=4, bias=False)
  )
  (experts): ParallelDroplessMLP(
    (mlp): SparseMLP()
  )
)
